Since the purpose of this tool is to look at User Made playlists, I wanted to grab playlists that are user made. Playlists.net has a database of user made playlists. I scraped their site for uri's. I kept it simple, if there was an error, I simply ignored the playlist rather than try and figure out why it wasn't working. Since this was just to grab data, it seemed better to grab data and move on, rather than spend a good deal of time trying to make it work. 

In [73]:
# imports
import requests
from bs4 import BeautifulSoup
import json
import csv
import pandas as pd
import spotipy
import spotipy.util as util
import types

In [69]:
url = 'https://playlists.net/annual-charts'

response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, "html.parser")

links = soup.findAll('td', attrs={'class': 'text-center'})

listing = []
for link in links:
    try:
        link_split = str(link.a).split('href="')[1].split('">')[0]
        create = 'https://playlists.net' + link_split
        listing.append(create)
    except:
        None

spotify_list = []
for url in listing:
    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    
    try:
        spotify = soup.findAll('div', attrs={'class':'pl-playlist-spotify-buttons'})
        spotify_uri = str(spotify).split('playlist:')[1].split('">')[0]
        spotify_list.append(spotify_uri)
    except:
        None

6BZDOi5mBuZvSySrNXobzS


In [70]:
print(spotify_list)

['3F8nzauyKh3YMqkx7nUO3l', '2FNVMVvyDvEumO8mbkHBxK', '3NlxYlzZ2Mb4mLVryphp4B', '020Zx6tGhxrMxW2E74VzG9', '3P6iaNsrF5cR8Rjpl2xPDE', '5137IgxwxPv1dXfsQkMMuE', '4uXqD5E8x8R8Ov8AKwVxhG', '6kXk9fAy9JCq8N1gwrbxDy', '4cs9OsFKYQ0r7wwd5hmjpc', '04fzbxkoRZW0jDbu7lkIwf', '2YhDK5qfwiP1dzmVZacFSU', '61RMzseNKapGbR6SK7uMsC', '7hJTW7aju8HizgmsY5ii5a', '77ctg4oGx88es5IUMGwfTJ', '5aJKNPKeumTZaezIXSDGwu', '6p5UpYDojTZ2poywfmvbqX', '0msFvwkXufQPjj4vnhoJps', '7m3aaTxCjuq7e3RXMl5Q9L', '41DyiTHHrmCRj3zJ4d4tjy', '428zCMVCQGMmbCCYawqsBV', '5g3AJrSTgmzsiVfs5N4nHp', '5WoEmxyYczjCm1CiwD1k2r', '2OFfgjs6kj0eA6FNayhAAJ', '2ofdCLICHquwOTgPJfZsZO', '0ETvdb3FNttb9UPFMcaS31', '7oicbXX0l8guqxjH5qij1q', '0tsu9axF3CsYD2ADxiRXK2', '2gMjVOInaVSLV40KGY5vEV', '1WzqNndeANO23OMnIrwTk5', '0dyiaySMt75rD8XERaDvbV', '6RYbFVAiSJ14OudXtpHr4I', '5Rrf7mqN8uus2AaQQQNdc1', '5gTfk1u2HwgwbwwcBibwRc', '7KNDpWQy3qMthJWXNDQutM', '4kae6PxGzDGVnI73KCKMrI', '0qQ0EZQGEHddZafsSf6JtX', '0i0N9QNaZao3RBv0t4g5uW', '6BYF7TZIFGQgi7OEHFExjg', '0V66wMHtVX

In [74]:
# authorization
token = util.prompt_for_user_token(
        username="rosey.waters@gmail.com",
        scope="user-read-recently-played user-read-private user-top-read user-read-currently-playing",
        client_id="e7b1f1096c82471d89d1db8030b82dd9",
        client_secret="5e635c87338b4729bc9a8bd007c2cbe1",
        redirect_uri="https://www.google.it/")

spotify = spotipy.Spotify(auth=token)

In [86]:
total = []
for i in spotify_list:
    playlist = 'https://api.spotify.com/v1/playlists/' + i
    
    try:
        def get_playlist(self, limit=1):
            return self._get(playlist, limit=limit)

        spotify.get_playlist = types.MethodType(get_playlist, spotify)

        playlist = spotify.get_playlist(limit=50)

        data = json.dumps(playlist, sort_keys=True, indent=2)

        data_parse = json.loads(data)

        description = data_parse['description']
        uri = data_parse['id']
        title = data_parse['name']
        tracks = data_parse['tracks']['total']
        
        lists = [description, uri, title, tracks]
        total.append(lists)
    except:
        None
        

In [87]:
playlists = pd.DataFrame(data=total, columns = ['description','spotify_id','name','no_tracks']) 

In [88]:
playlists.head()

,description,spotify_id,name,no_tracks
0,Ultimate list of head-boppers. - Submit songs...,3F8nzauyKh3YMqkx7nUO3l,The Very Best You Haven't Heard About,392
1,,2FNVMVvyDvEumO8mbkHBxK,Pitchfork 100 best tracks 2015 - detOkända.nu,92
2,Buzz Tracks TOP 100 for 2015,3NlxYlzZ2Mb4mLVryphp4B,2015 - The 100,101
3,,020Zx6tGhxrMxW2E74VzG9,Not Your Average Christmas,41
4,,3P6iaNsrF5cR8Rjpl2xPDE,100 Best New Artist 2015,105


In [89]:
playlists.to_csv('playlists2.csv')